# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [1]:
! git clone https://github.com/Redazh/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 5.00 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [2]:
mySeed = 200

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [3]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

une méthode simple consiste à utiliser une régression linéaire pour estimer ces coefficients.

In [13]:
from sklearn.linear_model import LinearRegression
# Extraction des prédicteurs (inputs) et des cibles (targets)
X_train = train_set['inputs']  # Shape: (n_samples, 3)
T_train = train_set['targets']  # Shape: (n_samples,)

# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, T_train)

# Extraction des coefficients estimés
Theta_linear_0 = model.intercept_  # theta_0
Theta_linear_rest = model.coef_    # Coefficients theta_1, theta_2, theta_3

print("Theta_0 :", Theta_linear_0)
print("Coefficients (theta_1, theta_2, theta_3) :", Theta_linear_rest)


Theta_0 : 10.078764034363882
Coefficients (theta_1, theta_2, theta_3) : [1.95156862 1.94842221 3.59966699]


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

Architecture adaptée :

*   Une seule couche linéaire prenant en entrée x,y,z et produisant une sortie t.
*   Pas besoin d'activation non linéaire (comme ReLU), car le modèle est linéaire.




*   Expressivité : Le modèle doit être capable d'apprendre une relation linéaire entre les prédicteurs (x,y,z) et les cibles (t). Une architecture avec une seule couche entièrement connectée (nn.Linearnn.Linear) est suffisante pour capturer cette relation.
*   Performances en généralisation : Une architecture simple limite le risque de surapprentissage (overfitting) et est bien adaptée à des données générées selon un modèle linéaire.





In [14]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)
import torch.nn as nn
# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Une seule couche linéaire (entrée 3 -> sortie 1)
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [15]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()

        outputs = mySimpleNet(batch_inputs)
        loss = criterion(outputs, batch_targets.unsqueeze(1))

        loss.backward()       # Compute gradients
        optimizer.step()      # Update model parameters

    # Print the loss for every 50 epochs
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [50/500], Loss: 3.6527
Epoch [100/500], Loss: 5.2404
Epoch [150/500], Loss: 3.7363
Epoch [200/500], Loss: 3.2035
Epoch [250/500], Loss: 3.4469
Epoch [300/500], Loss: 3.6447
Epoch [350/500], Loss: 3.4577
Epoch [400/500], Loss: 3.6525
Epoch [450/500], Loss: 3.8471
Epoch [500/500], Loss: 4.2082


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [16]:
Theta_neural_rest = mySimpleNet.linear.weight.data.numpy().flatten()  # Poids : theta_1, theta_2, theta_3
Theta_neural_0 = mySimpleNet.linear.bias.data.numpy().item()            # Biais : theta_0

print("Theta_0 (biais) :", Theta_neural_0)
print("Theta_1, Theta_2, Theta_3 :", Theta_neural_rest)

Theta_0 (biais) : 10.078777313232422
Theta_1, Theta_2, Theta_3 : [1.9517587 1.9488926 3.599925 ]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [22]:
from sklearn.metrics import mean_squared_error

# Jeu de test
X_test = test_set['inputs']  # Prédicteurs
T_test = test_set['targets']  # Cibles



# Calcul des prédictions
T_pred_linear = Theta_linear_0 + X_test @ Theta_linear_rest


# Calcul des prédictions
T_pred_neural = Theta_neural_0 + X_test @ Theta_neural_rest

# --- 3. Calcul des erreurs quadratiques moyennes ---
mse_linear = mean_squared_error(T_test, T_pred_linear)
mse_neural = mean_squared_error(T_test, T_pred_neural)

print(f"MSE Régression linéaire avec mean_squared_error: {mse_linear:.6f}")
print(f"MSE Réseau de neurones avec mean_squared_error: {mse_neural:.6f}")



MSE Régression linéaire avec mean_squared_error: 4.009261
MSE Réseau de neurones avec mean_squared_error: 4.009149


In [21]:
# Conversion des données en tenseurs PyTorch
T_test_tensor = torch.tensor(T_test, dtype=torch.float32)  # Cibles réelles
T_pred_linear_tensor = torch.tensor(T_pred_linear, dtype=torch.float32)  # Prédictions régression linéaire
T_pred_neural_tensor = torch.tensor(T_pred_neural, dtype=torch.float32)  # Prédictions réseau de neurones

# Calcul de la MSE avec `criterion`
mse_linear = criterion(T_pred_linear_tensor, T_test_tensor).item()
mse_neural = criterion(T_pred_neural_tensor, T_test_tensor).item()

print(f"MSE Régression linéaire (avec criterion): {mse_linear:.6f}")
print(f"MSE Réseau de neurones (avec criterion): {mse_neural:.6f}")


MSE Régression linéaire (avec criterion): 4.009261
MSE Réseau de neurones (avec criterion): 4.009149


Les deux méthodes donnent des performances quasiment identiques.

\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [26]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

Ce réseau est un Fully Convolutional Network (FCN) avec des convolutions causales adaptées aux séries temporelles.

In [37]:
print(model.down1)

Down_causal(
  (mpconv): Sequential(
    (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Double_conv_causal(
      (conv1): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)


In [35]:
# Nb de paramètres dans self.Down1: (calcul "à la main")

# Dimensions pour la première convolution
in_channels_1 = 64
out_channels_1 = 2*64
kernel_size_1 = 3

# Calcul des paramètres pour la première convolution
conv1_params = (in_channels_1 * kernel_size_1 + 1) * out_channels_1
print(f"Conv1 params = ({in_channels_1} * {kernel_size_1} + 1) * {out_channels_1} = {conv1_params}")

# BatchNorm pour la première convolution
bn1_params = 2 * out_channels_1
print(f"BatchNorm1 params = 2 * {out_channels_1} = {bn1_params}")

# Dimensions pour la deuxième convolution
in_channels_2 = 128
out_channels_2 = 128
kernel_size_2 = 3

# Calcul des paramètres pour la deuxième convolution
conv2_params = (in_channels_2 * kernel_size_2 + 1) * out_channels_2
print(f"Conv2 params = ({in_channels_2} * {kernel_size_2} + 1) * {out_channels_2} = {conv2_params}")

# BatchNorm pour la deuxième convolution
bn2_params = 2 * out_channels_2
print(f"BatchNorm2 params = 2 * {out_channels_2} = {bn2_params}")

# Total des paramètres pour Double_conv_causal
total_params = conv1_params + bn1_params + conv2_params + bn2_params
print(f"Total params in Double_conv_causal = {conv1_params} + {bn1_params} + {conv2_params} + {bn2_params} = {total_params}")






Conv1 params = (64 * 3 + 1) * 128 = 24704
BatchNorm1 params = 2 * 128 = 256
Conv2 params = (128 * 3 + 1) * 128 = 49280
BatchNorm2 params = 2 * 128 = 256
Total params in Double_conv_causal = 24704 + 256 + 49280 + 256 = 74496
Nombre total de paramètres dans le réseau : 2872641


In [41]:
# Nb de paramètres au total:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"Nombre total de paramètres dans le réseau : {total_params}")

Nombre total de paramètres dans le réseau : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

La taille du vecteur d'entrée est réduite par MaxPool1d et les convolutions avec stride dans les couches Down_causal. Elle est restituée par des convolutions transposées dans Up_causal et des skip connections qui combinent les informations locales et globales.

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

Pour augmenter le champ réceptif, le réseau utilise les mécanismes suivants :

*    Convolutions dilatées dans Double_conv_causal, où le paramètre dilation agrandit l'espacement entre les valeurs du noyau sans augmenter le nombre de paramètres.
*    Taille des noyaux (kernel_size) : Les noyaux kernel_size=3kernel_size=3 augmentent progressivement le champ réceptif.
*    Padding causal : Ajouté par F.pad pour maintenir la causalité tout en préservant la taille de la séquence.

In [46]:
print(model.inc)

Double_conv_causal(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [47]:
# Initialisation des paramètres
kernel_size1 = 3  # Taille du noyau de conv1
dilation1 = 1     # Dilatation de conv1
stride1 = 1       # Stride de conv1

kernel_size2 = 3  # Taille du noyau de conv2
dilation2 = 1     # Dilatation de conv2
stride2 = 1       # Stride de conv2

# Champ réceptif pour la première convolution
receptive_field1 = kernel_size1
print(f"Champ réceptif après conv1 = {receptive_field1}")

# Champ réceptif pour la deuxième convolution
receptive_field2 = kernel_size2 + (receptive_field1 - 1) * dilation2
print(f"Champ réceptif après conv2 = {receptive_field2}")

# Résultat final
print(f"Champ réceptif en sortie de self.inc = {receptive_field2}")


Champ réceptif après conv1 = 3
Champ réceptif après conv2 = 5
Champ réceptif en sortie de self.inc = 5


**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [49]:
# Initialisation du modèle
model.eval()  # Mode évaluation pour désactiver les dropouts

# Création de deux entrées identiques
input_length = 10000
input_tensor1 = torch.zeros(1, 1, input_length)
input_tensor2 = input_tensor1.clone()

# Modification d'une composante unique dans input_tensor2
index_to_modify = 5000
input_tensor2[0, 0, index_to_modify] = 1.0

# Passage des deux entrées dans le modèle
output1 = model(input_tensor1)
output2 = model(input_tensor2)

In [54]:
# Passage des deux entrées dans le modèle
model.eval()  # Mode évaluation (désactive le dropout, etc.)
with torch.no_grad():
    output1 = model(input_tensor1)  # Sortie associée à input_tensor1
    output2 = model(input_tensor2)  # Sortie associée à input_tensor2

# Recherche des composantes affectées
affected_indices = torch.nonzero((output1 - output2).abs() > 1e-6).squeeze()

# Affichage des résultats
print(f"Indices affectés dans la sortie : {affected_indices.tolist()}")
print(f"Taille empirique du champ réceptif pour y_5000 : {affected_indices.max().item() - affected_indices.min().item() + 1}")

Indices affectés dans la sortie : [[0, 0, 5000], [0, 0, 5001], [0, 0, 5002], [0, 0, 5003], [0, 0, 5004], [0, 0, 5005], [0, 0, 5006], [0, 0, 5007], [0, 0, 5008], [0, 0, 5009], [0, 0, 5010], [0, 0, 5011], [0, 0, 5012], [0, 0, 5013], [0, 0, 5014], [0, 0, 5015], [0, 0, 5016], [0, 0, 5017], [0, 0, 5018], [0, 0, 5019], [0, 0, 5020], [0, 0, 5021], [0, 0, 5022], [0, 0, 5024]]
Taille empirique du champ réceptif pour y_5000 : 5025


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



In [51]:
# Création de deux entrées identiques
input_length = 10000
input_tensor1 = torch.zeros(1, 1, input_length)  # Entrée initiale
input_tensor2 = input_tensor1.clone()

# Modifier uniquement les composantes t > 5000
input_tensor2[0, 0, 5001:] = 1.0  # Perturber les composantes après t=5000

# Passage dans le modèle
output1 = model(input_tensor1)  # Sortie pour l'entrée initiale
output2 = model(input_tensor2)  # Sortie pour l'entrée perturbée

# Vérifier si y5000 est affecté
if torch.allclose(output1[0, 0, 5000], output2[0, 0, 5000]):
    print("y5000 ne dépend pas des composantes xt avec t > 5000 (causalité respectée).")
else:
    print("y5000 dépend des composantes xt avec t > 5000 (causalité violée).")


y5000 ne dépend pas des composantes xt avec t > 5000 (causalité respectée).


La causalité est assurée par l'utilisation de padding causal dans la méthode forward de Double_conv_causal : `x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))`


*   F.pad ajoute des zéros uniquement à gauche de la séquence `((kernel_size−1)*dilation(kernel_size−1)×dilation)`.Cela garantit que la convolution ne "voit" que les éléments de xt​ pour t≤t_actuel​.
*   e padding causal est appliqué avant chaque convolution dans Double_conv_causal.



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

Dans le contexte du Learning to Rank (LTR), les auteurs font référence à :

*    Positive samples : Des documents ou items qui sont pertinents par rapport à une requête donnée.
*    Negative samples : Des documents ou items qui sont non pertinents ou moins pertinents par rapport à une requête donnée.

Prenons un moteur de recherche comme exemple :

*    Une requête utilisateur : "Meilleures recettes de gâteau au chocolat".
*.   Ensemble de résultats potentiels :
        Une page contenant une recette de gâteau au chocolat (pertinent → positive sample).
        Une page sur des gâteaux aux fruits (moins pertinent → negative sample).
        Une page sur la météo (non pertinent → negative sample).

Le modèle LTR vise à classer les documents de manière à ce que les positive samples soient classés plus haut que les negative samples dans les résultats.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

Dans l'expression de LRankNetLRankNet​, les zizi​ proviennent du modèle de scoring utilisé pour attribuer un score à chaque document (ou item) en fonction de sa pertinence vis-à-vis d'une requête donnée.


zi​ est le score de pertinence prédite pour un document ii par le modèle.
Ces scores sont utilisés pour comparer les documents entre eux dans une approche pairwise (par paires).
Un score plus élevé (zi>zj​) signifie que le modèle estime que le document i est plus pertinent que le document j.

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

L'expression de LRankNetLRankNet​ pousse le modèle à maximiser la probabilité σ(zi−zj), où zi​ est le score d'un positive sample et zj​ celui d'un negative sample.

En minimisant la perte, le modèle apprend à augmenter zi​ par rapport à zj​, ce qui garantit qu'après apprentissage, les scores des positive samples sont systématiquement supérieurs à ceux des negative samples pour chaque paire.

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?

Dans le cadre d'une approche par deep learning pour le Learning to Rank, on qualifie les réseaux de neurones comme des modèles pairwise (pairwise learning-to-rank models). La modalité d'entraînement est dite supervisée, car le modèle est entraîné à partir de paires de données avec des relations d'ordre explicites (positive > negative).